In [33]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools
import fasttext 

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd

In [4]:
editor = checklist.editor.Editor()
editor.tg

In [8]:
df = pd.read_csv('../test_cases/react-app/api/data/moderation/funny/projection_df')
text = df.comment
label = df.label

In [31]:
suite = TestSuite()


## MFTs

#### Politics

In [11]:
political_figures = ['Hillary Clinton', 'Donald Trump', 'Joe Biden', 'Kamala Harris', 'Mike Pence',
                     'Mitch McConnell', 'Nancy Pelosi', 'The President', 'The Vice President', 
                     'My Senator', 'My Representative']
editor.add_lexicon('political_figure', political_figures)

In [21]:
descriptive_nouns = editor.suggest('{political_figure} is {a:mask}')[:40]
editor.add_lexicon('descriptive_noun', descriptive_nouns)

In [27]:
adjectives = editor.suggest('{political_figure} is {a:mask} {descriptive_noun}')[:40]
editor.add_lexicon('adjective', adjectives)

In [28]:
t = editor.template('{political_figure} {be} {a:descriptive_noun}.', be=['is', 'was'], labels=1, save=True)
t += editor.template('{political_figure} {be} {a:adjective} {descriptive_noun}.', be=['is', 'was'], labels=1, save=True)

In [32]:
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 
          'Statements about political figures.', 
          'Vocabulary', 
          'Use descriptive nouns/adjectives to describe political figures, e.g., "Trump is a racist"')

In [54]:
t.data

['Hillary Clinton is an idiot.',
 'Hillary Clinton is an asshole.',
 'Hillary Clinton is a Nazi.',
 'Hillary Clinton is an extremist.',
 'Hillary Clinton is an atheist.',
 'Hillary Clinton is an Animal.',
 'Hillary Clinton is an American.',
 'Hillary Clinton is an embarrassment.',
 'Hillary Clinton is an anomaly.',
 'Hillary Clinton is an animal.',
 'Hillary Clinton is an alcoholic.',
 'Hillary Clinton is an anarchist.',
 'Hillary Clinton is an Arab.',
 'Hillary Clinton is an authoritarian.',
 'Hillary Clinton is an immigrant.',
 'Hillary Clinton is an enemy.',
 'Hillary Clinton is an Avenger.',
 'Hillary Clinton is an angel.',
 'Hillary Clinton is an assassin.',
 'Hillary Clinton is an actor.',
 'Hillary Clinton is an Alien.',
 'Hillary Clinton is an orphan.',
 'Hillary Clinton is an artist.',
 'Hillary Clinton is an alien.',
 'Hillary Clinton is an activist.',
 'Hillary Clinton is an ass.',
 'Hillary Clinton is an ape.',
 'Hillary Clinton is an Eagle.',
 'Hillary Clinton is an Icon.'

In [55]:
political_groups = ['liberals', 'conservatives', 'democrats', 'republicans', 
                   'the right', 'the left', 'the center right', 'the center left', 'the far right', 'the far left']
political_issue = ['taxes', 'healthcare', 'gun control', 'abortion', 'the supreme court', 'freedom of speech', 'censorship']

editor.add_lexicon('political_group', political_groups)
editor.add_lexicon('political_issue', political_issue)

In [63]:
adjectives = editor.suggest('{political_group} have {mask} views on {political_issue}')[:40]
editor.add_lexicon('political_adjective', adjectives)

In [71]:
t = editor.template('{political_group} {has} {adjective} views on {political_issue}.', has=['have', 'hold', 'push'], labels=1, save=True)
t += editor.template('{political_group} are {descriptive_noun}s.', labels=1, save=True)

In [75]:
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 
          'Statements about political groups.', 
          'Vocabulary', 
          'Use descriptive nouns/adjectives to describe political groups/views')

#### Complaining

In [86]:
nouns = ['traffic', 'weather', 'wait', 'lines', 'heat', 'experience', 'company', 'show', 'movie']
editor.add_lexicon('complain_noun', nouns, overwrite=True)

dislike = ['dislike', 'hate', 'abhor', 'can\'t stand']
editor.add_lexicon('dislike', dislike)

In [88]:
editor.suggest('The {complain_noun} is {mask}')[:50]
neg_adj = ['bad', 'brutal', 'terrible', 'awful', 'ugly', 'incomplete', 'unbearable', 'overwhelming', 'horrible']
pos_adj = ['good', 'great', 'awesome', 'excellent', 'incredible', 'amazing', 'fantastic', 'spectacular', 'interesting', 'beautiful', 'wonderful']
editor.add_lexicon('neg_adj', neg_adj, overwrite=True)
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)

In [89]:
t = editor.template('The {complain_noun} {be} {neg_adj}.', be=['is', 'was'], labels=1, save=True)
t += editor.template('I {dislike} the {complain_noun}, it\'s {neg_adj}.', labels=1, save=True)

In [91]:
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 
          'General complaining.', 
          'Vocabulary', 
          'Things described by negative adjectives or dislike.')

In [92]:
import re
def change_negative_adjective(x, *args, **kwargs):
    # Returns empty or a list of strings with profesions changed
    ret = []
    for a in neg_adj:
        if re.search(r'\b%s\b' % a, x):
            ret.extend([re.sub(r'\b%s\b' % a, b, x) for b in pos_adj])
    return ret

In [103]:
ret = Perturb.perturb(t.data, change_negative_adjective)
monotonic_decreasing = Expect.monotonic(label=1, increasing=False, tolerance=0.1)

In [104]:
test = DIR(**ret, expect=monotonic_decreasing)
suite.add(test, 
          'General complaining worse that positive comments.', 
          'Vocabulary', 
          'Things described by negative adjectives or dislike -> when switched to positive adjectives, p(mod) should decrease.')

In [110]:
test.run(predictionFunction, overwrite=True)
test.summary()

Predicting 5832 examples
Test cases:      486
After filtering: 89 (18.3%)
Fails (rate):    82 (92.1%)

Example fails:
0.8 I hate the wait, it's overwhelming.
1.0 I hate the wait, it's good.
0.9 I hate the wait, it's awesome.

----
0.8 The traffic is incomplete.
1.0 The traffic is good.
1.0 The traffic is awesome.

----
0.8 I hate the wait, it's terrible.
1.0 I hate the wait, it's good.
0.9 I hate the wait, it's awesome.

----


In [141]:
test.run(predictionFunction, overwrite=True)
test.summary()

Predicting 5832 examples
Test cases:      486
After filtering: 0 (0.0%)


## Run tests

In [111]:
model = fasttext.load_model('../test_cases/react-app/api/models/moderation/funny_model.bin')
model2 = fasttext.load_model('../test_cases/react-app/api/models/moderation/conspiracy_model.bin')

In [46]:
def getModelPredsHelper(allExamples, model):
    exampleList = [x.replace('\n', ' ') for x in allExamples]
    preds = model.predict(exampleList)
    preds_int = np.array([1 if 'positive' in p[0] else 0 for p in preds[0]])
    preds_prob = np.array([p[0] for p in preds[1]])
    probs = np.array([1 - p if preds_int[i] == 0 else p for (i,p) in enumerate(preds_prob)])
    probs = [np.round(p,3) for p in probs]
    return probs

def wrapSoftmax(probs):
    rounded_preds = np.round(probs)
    softmax = np.array([[p, 1-p] for p in probs])
    return rounded_preds, softmax

In [133]:
def predictionFunction(text):
    return wrapSoftmax(getModelPredsHelper(text, model))

def predictionFunctionModel2(text):
    return wrapSoftmax(getModelPredsHelper(text, model2))

In [134]:
suite.run(predictionFunction, overwrite=True)

Running Statements about political figures.
Predicting 36080 examples
Running Statements about political groups.
Predicting 8800 examples
Running General complaining.
Predicting 486 examples
Running General complaining worse that positive comments.
Predicting 5832 examples


In [135]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Statements about pol…

In [143]:
suite.run(predictionFunction, overwrite=True)

Running Statements about political figures.
Predicting 36080 examples
Running Statements about political groups.
Predicting 8800 examples
Running General complaining.
Predicting 486 examples
Running General complaining worse that positive comments.
Predicting 5832 examples


In [144]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Statements about pol…